In [ ]:
import os
import re
import numpy as np
from tqdm import tqdm
from scipy.signal import savgol_filter

def load_csv_data(directory, csv_files):
    return np.concatenate([
        np.loadtxt(os.path.join(directory, file), delimiter=',')
        for file in csv_files
    ])

def extract_start_number(filename):
    match = re.search(r'F(\d+)_', filename)
    return int(match.group(1)) if match else 0

def noise_filter(data, window_size):
    return savgol_filter(data, window_size, 2, deriv=0, delta=1.0, axis=-1, mode='interp', cval=0.0)


def find_clusters(arr, window_size=500, threshold=0.3):
    clusters = []
    in_cluster = False
    start, end = None, None

    for i in range(len(arr) - window_size + 1):
        window = arr[i:i + window_size]
        non_nan_count = np.sum(~np.isnan(window))
        if non_nan_count / window_size > threshold:
            if not in_cluster:
                start = next(j for j, v in enumerate(window) if not np.isnan(v)) + i
                in_cluster = True
        else:
            if in_cluster:
                end = next(j for j, v in enumerate(window[::-1]) if not np.isnan(v))
                end = i + window_size - end - 1
                clusters.append((start, end))
                in_cluster = False
 
    if in_cluster:
        clusters.append((start, len(arr) - 1))

    return clusters


def find_largest_cluster(bounds):
    if not bounds:
        return None
    return max(bounds, key=lambda x: x[1] - x[0])


def truncate_after_clusters(arr, bounds, min_dist=5000):
    if not bounds:
        return arr

    largest_bound = find_largest_cluster(bounds)
    last_element = largest_bound[1]
    
    for start, end in bounds:
        if start <= last_element + min_dist and end > last_element:
            last_element = end

    arr[last_element + 1:] = np.nan
    return arr


def find_valid_true(mask, window_size, threshold):
    mask = np.asarray(mask, dtype=bool)
    true_indices = np.where(mask)[0]
    
    for idx in true_indices:
        end = idx + window_size
        if end > len(mask):
            break
        window = mask[idx:end]
        if np.mean(window) > threshold:
            return idx
    return np.nan 
    

def process_directory(directory, mode):
    if not os.path.exists(directory):
        return np.nan
        
    csv_files = sorted(
        [file for file in os.listdir(directory) if file.endswith('.csv') and 'area' in file],
        key=extract_start_number
    )
    data = load_csv_data(directory, csv_files)
    
    if mode == "hatch":
        data[data <= 8] = np.nan                      # remove noise below 8, can be adjusted
        weighted_area = noise_filter(data, 10)        # smooth the data with a window size of 10, can be adjusted
        mask = (weighted_area != 0) & ~np.isnan(weighted_area)
        
    elif mode == "pupation":
        median_value = np.nanmedian(data)
        
        if median_value < 100:                        # remove noise below 100, can be adjusted
            data[data <= median_value] = np.nan
        else:
            data[data <= 100] = np.nan
          

        data = truncate_after_clusters(data, find_clusters(data))
        weighted_area = noise_filter(data, 20)        # smooth the data with a window size of 20, can be adjusted
        mask = (weighted_area != 0) & ~np.isnan(weighted_area)

    elif mode == "eclosion":
        mean_value = np.nanmean(data)

        if mean_value < 500:                          # remove noise below 500, can be adjusted
            data[data <= mean_value] = np.nan
        else:
            data[data <= 500] = np.nan
            
        weighted_area = noise_filter(data, 45)        # smooth the data with a window size of 45, can be adjusted
        mask = (weighted_area != 0) & ~np.isnan(weighted_area)
        
        
    else:
        raise ValueError("mode must be 'hatch' or 'pupation' or 'eclosion'")

    
    if np.any(mask):
        if mode == "hatch":
            return np.where(mask)[0][0]
        elif mode == "pupation":
            return np.where(mask)[0][-1]
        elif mode == "eclosion":
            true_first_index = find_valid_true(mask, 1800, 0.2)  # adjust threshold as needed
            return true_first_index
    
    return np.nan



times = np.empty((96, 4), dtype=object)
times[:, 1:] = np.nan                                 # initialize hatch & pupation & eclosion times as nan


index = 0
for letter in tqdm('ABCDEFGH'):
    for number in range(1, 13):
        well_name = f"{letter}{number}"
        hatch_dir = f"/directory/containing/area/csv/files/{well_name}"  # directory containing the hatching area csv files of each well
        pupation_dir = f"/directory/containing/area/csv/files/{well_name}" # directory containing the pupation area csv files of each well
        eclosion_dir = f"/directory/containing/area/csv/files/{well_name}"  # directory containing the eclosion area csv files of each well

        if not os.path.isdir(hatch_dir) or not os.listdir(hatch_dir):
            hatch_time = np.nan
        else:
            hatch_time = process_directory(hatch_dir, "hatch")
            
        if not os.path.isdir(pupation_dir) or not os.listdir(pupation_dir) or np.isnan(hatch_time):
            pupation_time = np.nan
        else:
            pupation_time = process_directory(pupation_dir, "pupation")

        if not os.path.isdir(eclosion_dir) or not os.listdir(eclosion_dir) or np.isnan(hatch_time):
            eclosion_time = np.nan
        else:
            eclosion_time = process_directory(eclosion_dir, "eclosion")
            
        
        times[index] = [well_name, hatch_time, pupation_time, eclosion_time]
        print(times[index])
        index += 1

